In [1]:
!pip install langchain faiss-cpu sentence-transformers transformers

In [2]:
from google.colab import files
uploaded_files = files.upload()
file_list = list(uploaded_files.keys())


Saving formatted_rasa_data_flat.csv to formatted_rasa_data_flat (6).csv
Saving formatted_products_spicezone.csv to formatted_products_spicezone (6).csv
Saving formatted_products_nasha.csv to formatted_products_nasha (6).csv
Saving formatted_products_chaiatacos.csv to formatted_products_chaiatacos (6).csv
Saving formatted_products_cafespice.csv to formatted_products_cafespice (6).csv


In [3]:
import pandas as pd

def load_items_and_restaurants_from_csv(file_list):
    item_documents = []
    restaurant_names = set()

    for file_name in file_list:
        df = pd.read_csv(file_name)

        for index, row in df.iterrows():
            # Cleanly read all fields
            restaurant = str(row.get('restaurant', '') or '').strip()
            category = str(row.get('category', '') or '').strip()
            item_name = str(row.get('item_name', '') or '').strip()
            description = str(row.get('description', '') or '').strip()
            options = str(row.get('options', '') or '').strip()
            tags = str(row.get('tags', '') or '').strip()
            spice_level = str(row.get('spice_level', '') or '').strip()
            state = str(row.get('state', '') or '').strip()
            address = str(row.get('address', '') or '').strip()
            contact_info = str(row.get('contact_info', '') or '').strip()

            restaurant_names.add(restaurant)

            # Build enriched document
            doc = (
                f"Restaurant: {restaurant}. "
                f"Category: {category}. "
                f"Item: {item_name}. "
                f"Description: {description}. "
                f"Options: {options}. "
                f"Tags: {tags}. "
                f"Spice Level: {spice_level}. "
                f"Location: {state}, {address}. "
                f"Contact Info: {contact_info}."
            )
            item_documents.append(doc)

    # Restaurant documents separately
    restaurant_documents = [f"Restaurant Name: {r}" for r in restaurant_names if r]

    return item_documents, restaurant_documents


In [4]:
pip install langchain-experimental

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS databases
item_db = FAISS.from_texts(item_documents, embedding=embedding_model)
restaurant_db = FAISS.from_texts(restaurant_documents, embedding=embedding_model)


<ipython-input-5-0a0aae609a8d>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models 

NameError: name 'item_documents' is not defined

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.llms import HuggingFacePipeline

model_name = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

hf_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.5,
    repetition_penalty=1.3
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Custom prompt for full sentence answers
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful assistant for a restaurant.

Use complete sentences and human-like responses.

You provide correct and full/complete answers.

Context:
{context}

Question:
{question}

Answer:
"""
)

# Item Chain
item_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=item_db.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    chain_type_kwargs={"prompt": custom_prompt},
    return_source_documents=True
)



In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def chatbot(query):
    result = item_chain.invoke(query)
    final_answer = result["result"]
    sources = "\n".join([doc.page_content for doc in result["source_documents"]])
    return f"🍽️ Answer:\n{final_answer}\n\n📋 Sources:\n{sources}"


custom_theme = gr.themes.Base(
    primary_hue="rose",
    secondary_hue="red", ).set(
    body_background_fill="#FAFAFA",
    button_primary_background_fill="#E23744",
    button_primary_text_color="#FFFFFF",
    button_primary_background_fill_hover="#A6192E",
    # text_color="#2C2C2C",
    input_background_fill="#FFFFFF",
    input_border_color="#E23744",
)

gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask me about the menu..."),
    outputs="text",
    title="Alergen and Food Info Chatbot",
    description="Ask about categories, items, allergens, vegetarian options and more!",
    theme=custom_theme,
).launch(share=True)
